In [1]:
# 1. 문서 로드

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('/data/ephemeral/home/work/python/gx-rag/src/apps/persona/data/source', glob="*", show_progress=True)
docs = loader.load()

100%|██████████| 5/5 [00:03<00:00,  1.31it/s]


In [2]:
type(docs[0])

len(docs)

5

In [3]:
# 2. 문서 의미 있게 자르기 (1, 2, 3,4 ...장 별로)

import re
from langchain.schema import Document

pattern = r'(?=\d+\.\s)'  # 
#pattern = r'(?=^\d{1,2}\.\s)'  # Positive Lookahead 사용
#pattern = r'(?=[0-9]{1,2}\.\s)'    # ASCII 2자리
#pattern = r'(?=[1-9]\d?\.\s)'  # 1-9 또는 10-99

all_text = "\n\n".join([doc.page_content for doc in docs])
chunks = re.split(pattern, all_text, flags=re.MULTILINE)
# 무효한 리스트 삭제하기
valid_chunks = [chunk.strip() for chunk in chunks if len(chunk.strip()) > 100 and not chunk.strip().isdigit()]
len(valid_chunks)

valid_docs = [Document(page_content=chunk, metadata={'level': 'parent'}) for chunk in valid_chunks]

In [4]:
len(valid_docs)

46

In [5]:
# 2-1  내용 확인(test)
for doc in valid_docs:
    print("*"*50)
    print(doc.page_content[:100])

**************************************************
1. 없어진 비밀 서류..

1895년 11월 세째 주, 런던은 짙은 안개로 덮여 있었다. 월요일부터 목요일에 걸 쳐서는 베이커 가의 우리 집 창문에서 길 건너편 집들의 모습이 보
**************************************************
2. 세 개의 열쇠..

한 시간 뒤, 홈즈와 레스트레이드와 나는 올드게이트 역 바로 앞에서, 지하철이 터널 밖으로 나오는 지점에 서 있었다. 불그스름한 얼굴의 점잖은 노신사가 지
**************************************************
3. 은신처..

런던의 베이커 가로 돌아와 보니 아니나다를까 편지 한 통이 와 있었다. 정부의 문서 배달원이 속달로 배달한 것이었다. 홈즈는 대충 읽고 나서 내게 던져 주었 다.
**************************************************
4. 검은 그림자..

다음날 아침, 식사가 끝나자 약속한 대로 마이크로프트 홈즈와 레스트레이드 경 감이 찾아오자, 셜록 홈즈는 우리가 어제 한 행동을 설명해 주었다.우리가 강도 
**************************************************
1. 정신병자의 짓인가?..

레스트레이드 경감은 날씨라든가 신문에 난 기사에 대해서 띄엄띄엄 이야기를 하더니, 갑자기 뭔가를 골똘히 생각하는 표정을 지었다.   그래서 홈즈가 이
**************************************************
2. 이상한 살인..

다음날 아침, 내가 겨우 잠에서 깨어났을 때 홈즈가 허둥지둥 달려왔다. 손에는 전보를 들고 있었다. - 켄징턴 구 피트 거리 131번지로 즉시 오시오. 레스
**************************************************
3. 나폴레옹 흉상의 출처...

홈즈와 나는 하이 거리에 있는 하

In [6]:
# 2-2 사이즈 확인(test)
def get_chunk_size_by_words(docs):
    
    for doc in docs:
        print(len(doc.split()))

get_chunk_size_by_words(valid_chunks)

2080
2209
1334
1280
717
1305
1227
490
855
695
528
561
818
694
1122
1182
563
1487
1539
1674
2707
1627
1445
2168
2026
1987
1256
934
2110
1932
2699
1357
706
720
606
1319
938
1094
1287
1080
1725
926
397
1025
946
1068


In [ ]:
# 3. 디비에 저장

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import LocalFileStore, create_kv_docstore
import faiss
from langchain_community.vectorstores import FAISS

from langchain_huggingface import HuggingFaceEmbeddings

import torch

embed_model=HuggingFaceEmbeddings(
                                    model_name="BAAI/bge-m3",
                                    model_kwargs={"device": "cuda"} ,
                                    encode_kwargs={"normalize_embeddings": True})


""" # HuggingFaceEmbeddings 인스턴스 생성
embed_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-8B",
    model_kwargs={
        "device": "cpu"  # GPU 사용 (CPU는 "cpu")
    },
    encode_kwargs={
        "normalize_embeddings": True  # 임베딩 정규화
    }
) """

# 1. 컴포넌트 설정
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)  # 작은 청크
#vectorstore = Chroma(embedding_function=embeddings)  # 벡터 저장소

# 빈 벡터스토어 생성
vectorstore = FAISS.from_texts(
    texts=["초기화"],  # 더미 텍스트
    embedding=embed_model
)

#store = InMemoryStore()
# doc스토어 시스템에 영구 저장
fs = LocalFileStore(root_path="/data/ephemeral/home/work/python/gx-rag/src/apps/persona/data/store")
store = create_kv_docstore(fs) 

# 2. ParentDocumentRetriever 생성
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,    # 청크 저장 (검색용)
    docstore=store,            # 원본 저장 (반환용)  
    child_splitter=child_splitter,  # 분할 방법
    search_kwargs={"k": 1} 
)

# 3. 문서 추가
retriever.add_documents(valid_docs)

# 벡터스토어 저장
vectorstore.save_local('/data/ephemeral/home/work/python/gx-rag/src/apps/persona/data/faiss')

In [ ]:
result= retriever.get_relevant_documents('은성호 사건을 해결할수 있었던 힌트는?')
print(result[0].page_content)
len(result)

In [ ]:
# 4.  요약 쿼리 템플릿

template = """
아래 <context> 내용을 요약해서 원본 크기의 30%정도의 크기로 만들어줘. 
<context> 에 있는 내용만 이야기해줘.
양식의 규격을 준수해줘.  

[example]

<context>
    나는 1878년 런던 대학에서 의학 박사 학위를 받고 군의관이 되기 위해 필요한 
자격을 따기 위해 네틀리 육군 병원에 들어갔다. 그곳에서 규정된 교육을 끝내고 
노섬벌랜드 연대 소속 군의관으로 임명되었다.
그 무렵 이 연대는 인도에 주둔하고 있었는데, 내가 현지에 도착하기도 전에 제2
차 아프가니스탄 전쟁이 일어났다. 나는 봄베이에 상륙해서야 나의 연대가 적진 
깊숙이로 공격해 들어간 것을 알았다. 그래서 길을 재촉하여 그 뒤를 쫓아가 ㄴ
무사히 연대를 만나 새로운 임무에 종사하게 되었던 것이다.
이 전쟁에서 나는 연거푸 재난과 마주쳐야 했다. 마이완드에서 격전에 참가했다
가 어깨에 탄환을 맞아 뼈가 으스러지고 동맥까지 다치게 되었던 것이다.
다행히 전령을 맡고 있던 부하 머레이가 충직하고 용감한 사람이어서, 나를 말 
위에 짐짝처럼 싣고 무사히 아군의 진지까지 데리고 돌아왔으니 망정이지, 만일 
그가 없었다면 나는 잔인한 적의 수중에 사로잡혔을 것이다.
나는 후방 병원으로 보내졌다. 그 곳에서 어느 정도 건강이 회복되어 병원 안을 
걸어다니기도 하고, 베란다에 나가 일광욕도 할 만큼 회복되었는데, 운수 사납게
도 이번에는 장티푸스에 걸리고 말았다.
</context>

<summary>
    나는 1878년 런던 대학에서 의학 박사 학위를 받고 네틀리 육군 병원에서 군의관 교육을 받은 후, 노섬벌랜드 연대 소속 군의관으로 임명되었다.
제2차 아프가니스탄 전쟁이 발발하여 인도로 파견된 나는 봄베이에 도착해서야 연대가 이미 전선으로 떠났음을 알고 급히 뒤를 쫓아 연대와 합류했다.
전쟁 중 마이완드 격전에서 어깨에 탄환을 맞아 뼈가 으스러지고 동맥까지 다쳤다. 다행히 부하 머레이가 나를 구해 후방 병원으로 이송되었고, 어느 정도 회복되었으나 이번엔 장티푸스에 걸렸다.
</summary>


[Generate]

<context>
    {context}
</context>

<summary>
    [Your summary here - NOTHING ELSE]
</summary>
"""

In [ ]:
# 5.  요약쿼리 테스트

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_ollama import ChatOllama

#llm = ChatOllama(model="phi4:latest")
llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")


prompt = ChatPromptTemplate.from_template(template)
holmes_chain = prompt | llm | StrOutputParser()


# 체인 실행 전에 프롬프트 미리 확인
""" formatted_prompt = prompt.format(context=valid_docs[2].page_content)
print("=== 실제 전달될 프롬프트 ===")
print(formatted_prompt)
print("=" * 50)  """


result = holmes_chain.invoke({'context': valid_docs[2].page_content})
print(result)